# Imports necessários


In [1]:
import pandas as pd
import numpy as np

# Import dos dados

In [3]:
# features receitas
dados_receitas = pd.read_csv('03_dados_carga/arquivos_para_carga/dados_receitas.csv', sep=';')
dados_receitas.head()

,COD_RECEITA,DATA,COD_CONTRIBUINTE,VALOR_ARRECADADO,FONTE_DADOS
0,1722010101,2013-01-03,886cd0eabf5a18,185560.68,prefeitura municipal
1,1722010101,2013-01-04,886cd0eabf5a18,8081.80,prefeitura municipal
2,1722010101,2013-01-07,886cd0eabf5a18,4231.39,prefeitura municipal
3,1722010101,2013-01-08,886cd0eabf5a18,10536.53,prefeitura municipal
4,1722010101,2013-01-15,886cd0eabf5a18,103118.27,prefeitura municipal


In [4]:
# features despesas
dados_despesas = pd.read_csv('03_dados_carga/arquivos_para_carga/novos_dados_despesas.csv', sep=';')
dados_despesas.head()

,DATA_FATO,COD_DESPESA,CREDOR,NUM_DOCUMENTO,FONTE_RECURSO,ORGAO_INTERNO,ORGAO_VINCULADO,ELEMENTO,SUBELEMENTO,FUNCAO,SUBFUNCAO,NATUREZA,VALOR_FIXADO,VALOR_EMPENHADO,VALOR_LIQUIDADO,VALOR_PAGO,SALDO
0,2013-12-31,3300,alianca do brasil seguros s a,01.378.407/0001-10,recursos ordinarios,secretaria municipal de infra estrutura,prefeitura municipal,39.0,2044.0,4,122,3,2911694.0,5120.0,5120.0,0.0,5120.0
1,2013-12-31,3299,ademir santana da silva,***.893.004-**,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36.0,2029.0,13,392,3,107000.0,6950.0,6950.0,0.0,6950.0
2,2013-12-31,3298,olindina glicia gomes pereira,***.859.974-**,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36.0,2029.0,13,392,3,107000.0,800.0,800.0,0.0,800.0
3,2013-12-31,3297,taise kelly cezario de souza,***.039.644-**,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36.0,2029.0,13,392,3,107000.0,400.0,400.0,0.0,400.0
4,2013-12-31,3296,otavio leoncio pereira,***.396.364-**,recursos ordinarios,secretaria municipal de educacao,prefeitura municipal,36.0,2029.0,13,392,3,107000.0,600.0,600.0,0.0,600.0


# Árvore de decisão